In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [3]:
df = pd.read_csv(r"C:\Users\tom_r\Desktop\crime data\Data-Science\TFL_cycling\data\411JourneyDataExtract01Jan2025-14Jan2025.csv")
df

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
0,145666816,2025-01-14 23:59,1043,"Museum of London, Barbican",2025-01-15 00:13,200149,"Watney Street, Shadwell",55223,CLASSIC,14m 30s,870468
1,145666817,2025-01-14 23:59,300015,"Binfield Road, Stockwell",2025-01-15 00:04,300229,"Sidney Road, Stockwell",20325,CLASSIC,5m 7s,307181
2,145666818,2025-01-14 23:59,1068,"Norton Folgate, Liverpool Street",2025-01-15 00:10,1051,"Shoreditch Park, Hoxton",50432,CLASSIC,10m 40s,640755
3,145666819,2025-01-14 23:59,1159,"Berry Street, Clerkenwell",2025-01-15 00:10,1007,"Drury Lane, Covent Garden",56794,CLASSIC,10m 42s,642548
4,145666812,2025-01-14 23:58,200048,"Page Street, Westminster",2025-01-15 00:14,1058,"Edgware Road Station, Marylebone",63546,PBSC_EBIKE,15m 44s,944185
...,...,...,...,...,...,...,...,...,...,...,...
213395,145449851,2025-01-01 00:02,1190444,"Kennington Lane Rail Bridge, Vauxhall_OLD",2025-01-01 00:05,1205,"Black Prince Road, Vauxhall",57206,CLASSIC,3m 16s,196134
213396,145449841,2025-01-01 00:01,200248,"Pitfield Street Central, Hoxton",2025-01-01 00:04,3445,"Shoreditch High Street, Shoreditch",40770,CLASSIC,3m 40s,220562
213397,145449838,2025-01-01 00:00,1107,"Gloucester Road Station, South Kensington",2025-01-01 00:03,22182,"South Parade, Chelsea",62229,PBSC_EBIKE,3m 10s,190839
213398,145449839,2025-01-01 00:00,1037,"Park Lane , Hyde Park",2025-01-01 00:20,1037,"Park Lane , Hyde Park",55679,CLASSIC,20m 13s,1213746


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213400 entries, 0 to 213399
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Number                213400 non-null  int64 
 1   Start date            213400 non-null  object
 2   Start station number  213400 non-null  int64 
 3   Start station         213400 non-null  object
 4   End date              213400 non-null  object
 5   End station number    213400 non-null  int64 
 6   End station           213400 non-null  object
 7   Bike number           213400 non-null  int64 
 8   Bike model            213400 non-null  object
 9   Total duration        213400 non-null  object
 10  Total duration (ms)   213400 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 17.9+ MB


In [5]:
# Converty start and end times to proper datatype
df['Start date'] = pd.to_datetime(df['Start date'])
df['End date'] = pd.to_datetime(df['End date'])

In [6]:
# convert objects to string
df['Start station'] = df['Start station'].astype('string')
df['End station'] = df['End station'].astype('string')
df['Bike model'] = df['Bike model'].astype('string')

# convert duration to timedelta 
df['Total duration (ms)'] = pd.to_numeric(df['Total duration (ms)'])

# drop unneeded column
df = df.drop(columns = 'Total duration')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213400 entries, 0 to 213399
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Number                213400 non-null  int64         
 1   Start date            213400 non-null  datetime64[ns]
 2   Start station number  213400 non-null  int64         
 3   Start station         213400 non-null  string        
 4   End date              213400 non-null  datetime64[ns]
 5   End station number    213400 non-null  int64         
 6   End station           213400 non-null  string        
 7   Bike number           213400 non-null  int64         
 8   Bike model            213400 non-null  string        
 9   Total duration (ms)   213400 non-null  int64         
dtypes: datetime64[ns](2), int64(5), string(3)
memory usage: 16.3 MB


In [8]:
print(df.isna().sum())

Number                  0
Start date              0
Start station number    0
Start station           0
End date                0
End station number      0
End station             0
Bike number             0
Bike model              0
Total duration (ms)     0
dtype: int64


In [11]:
DB_USER = "postgres"
DB_PASSWORD = ""  # populate when using
DB_HOST = "localhost" 
DB_PORT = "5432"
DB_NAME = "tfl_cycling"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
conn = engine.connect()

df.to_sql("journeys", engine, if_exists="replace", index=False)


400